In [13]:
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from osgeo import gdal
from scipy.io import loadmat
from scipy.fft import ifft, fft, fftshift, ifftshift, fftfreq
from scipy.fft import ifft2, fft2
from scipy.constants import pi, c

In [14]:
PRIMARY_HEADER = [
    3,   # Packet Version Number
    1,   # Packet Type
    1,   # Secondary Header Flag
    7,   # Process ID
    4,   # Process Category
    2,   # Sequence Flags
    14,  # Packet Sequence Count 
    16   # Packet Data Length
]

PRIMARY_HEADER_FIELDS = [
    'packet_version_number',
    'packet_type',
    'secondary_header_flag',
    'process_id',
    'process_category',
    'sequence_flags',
    'packet_sequence_count',
    'packet_data_length'
]

SECONDARY_HEADER = [
    32,  # Coarse Time
    16,  # Fine Time
    32,  # Sync Marker
    32,  # Data Take ID
    8,   # ECC Number
    1,   # N/A
    3,   # Test Mode
    4,   # RX Channel ID
    32,  # Instrument Configuration ID
    8,   # Sub-Commutative Ancillary Data Word Index
    16,  # Sub-Commutative Ancillary Data Word
    32,  # Counter Service
    32,  # PRI Count
    1,   # Error Flag
    2,   # N/A
    5,   # BAQ Mode
    8,   # BAQ Block Length
    8,   # N/A
    8,   # Range Decimation
    8,   # RX Gain
    16,  # TX Ramp Rate
    16,  # Pulse Start Frequency
    24,  # Pulse Length
    3,   # N/A
    5,   # Rank
    24,  # PRI
    24,  # SWST
    24,  # SWL
    24,  # SAS SSB Message *** TODO: See Page 43, 44
    24,  # SES SSB Message *** TODO: See Page 50
    16,  # Number of Quadratures
    8,   # N/A
]

SECONDARY_HEADER_FIELDS = [
    'coarse_time',
    'fine_time',
    'sync_marker',
    'data_take_id',
    'ecc_number',
    'na_1',
    'test_mode',
    'rx_channel_id',
    'instrument_configuration_id',
    'sc_data_word_index',
    'sc_data_word',
    'counter_service',
    'pri_count',
    'error_flag',
    'na_2',
    'baq_mode',
    'baq_block_length',
    'na_3',
    'range_decimation',
    'rx_gain',
    'tx_ramp_rate',
    'pulse_start_frequency',
    'pulse_length',
    'na_4',
    'rank',
    'pri',
    'swst',
    'swl',
    'sas_ssb_message',
    'ses_ssb_message',
    'num_quadratures',
    'na_5',
]

SUB_COMMUTATIVE_DATA_SERVICE = [
    16,      # Dummy Data
    64,      # X-Axis ECEF Position
    64,      # Y-Axis ECEF Position
    64,      # Z-Axis ECEF Position
    32,      # X-velocity ECEF
    32,      # Y-velocity ECEF
    32,      # Z-velocity ECEF
    16,      # POD Data Stamp 1
    16,      # Pod Data Stamp 2
    16,      # Pod Data Stamp 3
    16,      # Pod Data Stamp 4
    32,      # Q0 Attitude Quaternion
    32,      # Q1 Attitude Quaternion
    32,      # Q2 Attitude Quaternion
    32,      # Q3 Attitude Quaternion
    32,      # OmegaX Angular Rate
    32,      # OmegaY Angular Rate
    32,      # OmegaZ Angular Rate
    16,      # Data Time Stamp 1
    16,      # Data Time Stamp 2
    16,      # Data Time Stamp 3
    16,      # Data Time Stamp 4
    16,      # Pointing Status
    16,      # Temperature Update Status
    8, 8, 8, # Tile 1 EFE H, V Temperature and Activate TA Temperature
    8, 8, 8, # Tile 2
    8, 8, 8, # Tile 3
    8, 8, 8, # Tile 4
    8, 8, 8, # Tile 5
    8, 8, 8, # Tile 6
    8, 8, 8, # Tile 7
    8, 8, 8, # Tile 8
    8, 8, 8, # Tile 9
    8, 8, 8, # Tile 10
    8, 8, 8, # Tile 11
    8, 8, 8, # Tile 12
    8, 8, 8, # Tile 13
    8, 8, 8, # Tile 14
    9,       # N/A
    7,       # TGU Temperature
]

In [414]:
def create_bit_string(bytes_string: str):
    bit_string = ''
    for byte in bytes_string:
        bit_string += '{:08b}'.format(byte)
    return bit_string


read_and_pop = lambda bit_string, bit_length: (bit_string[0:bit_length], bit_string[bit_length:])


def bit_len_map(brc: int):
    k = 1
    match brc:
        case 0:
            k = 4
        case 1:
            k = 5
        case 2:
            k = 7
        case 3:
            k = 10
        case 4:
            k = 16
    return k


def huffman_4(bit_string):
    k = bit_len_map(4)
    m_code = 0
    bit, bit_string = read_and_pop(bit_string, 1)
    bit_len = 1
    if bit == '0':  # 0 <-
        bit, bit_string = read_and_pop(bit_string, 1)
        bit_len += 1
        if bit == '0':  # 0 <- 0 <-
            return 0, bit_string, bit_len
        else:
            bit, bit_string = read_and_pop(bit_string, 1)
            bit_len += 1
            return (1, bit_string, bit_len) if bit == '0' else (2, bit_string, bit_len)  # 0 or 1 <- 1 <- 0
    else:
        bit, bit_string = read_and_pop(bit_string, 1)
        bit_len += 1
        if bit == '0':
            bit, bit_string = read_and_pop(bit_string, 1)
            bit_len += 1
            return (3, bit_string, bit_len) if bit == '0' else (4, bit_string, bit_len)  # 0 or 1 <- 1 <- 0
        else:
            bit, bit_string = read_and_pop(bit_string, 1)
            bit_len += 1
            if bit == '0':
                bit, bit_string = read_and_pop(bit_string, 1)
                bit_len += 1
                return (5, bit_string, bit_len) if bit == '0' else (6, bit_string, bit_len)  # 0 or 1 <- 1 <- 0
            else:
                bit, bit_string = read_and_pop(bit_string, 1)
                bit_len += 1
                if bit == '0':
                    return 7, bit_string, bit_len
                else:
                    bit, bit_string = read_and_pop(bit_string, 1)
                    bit_len += 1
                    if bit == '0':
                        return 8, bit_string, bit_len
                    else:
                        bit, bit_string = read_and_pop(bit_string, 1)
                        bit_len += 1
                        if bit == '0':
                            return 9, bit_string, bit_len
                        else:
                            bit, bit_string = read_and_pop(bit_string, 1)
                            bit_len += 1
                            if bit == '0':
                                bit, bit_string = read_and_pop(bit_string, 1)
                                bit_len += 1
                                return (10, bit_string, bit_len) if bit == '0' else (11, bit_string, bit_len)
                            else:
                                bit, bit_string = read_and_pop(bit_string, 1)
                                bit_len += 1
                                if bit == '0':
                                    bit, bit_string = read_and_pop(bit_string, 1)
                                    bit_len += 1
                                    return (12, bit_string, bit_len) if bit == '0' else (13, bit_string, bit_len)
                                else:
                                    bit, bit_string = read_and_pop(bit_string, 1)
                                    bit_len += 1
                                    return (14, bit_string, bit_len) if bit == '0' else (15, bit_string, bit_len)
    


def huffman_3(bit_string):
    k = bit_len_map(3)
    m_code = 0
    bit, bit_string = read_and_pop(bit_string, 1)
    bit_len = 1
    if bit == '0':
        bit, bit_string = read_and_pop(bit_string, 1)
        bit_len += 1
        if bit == '0':
            return 0, bit_string, bit_len
        else:
            return 1, bit_string, bit_len
    else:
        m_code = 2
        for i in range(k - 3):
            bit, bit_string = read_and_pop(bit_string, 1)
            bit_len += 1
            if bit == '0':
                break
            else:
                m_code += 1
        return m_code, bit_string, bit_len


def huffman(bit_string: str, brc: int):
    if brc == 3:
        return huffman_3(bit_string)
    elif brc == 4:
        return huffman_4(bit_string)
    else:
        bit_len = 0
        k = bit_len_map(brc)
        m_code = 0
        for i in range(k - 1):
            bit, bit_string = read_and_pop(bit_string, 1)
            bit_len += 1
            if bit == '0':
                break
            else:
                m_code += 1
        return m_code, bit_string, bit_len


def get_s_codes(
    bit_string: str,
    brc: int,
) -> ([int], [int], str):
    signs = []
    m_codes = []
    s_code_count = 0
    total_bits = 0
    while s_code_count < 128:
        sign, bit_string = read_and_pop(bit_string, 1)
        m_code, bit_string, bit_len = huffman(bit_string, brc)
        signs.append(sign)
        m_codes.append(m_code)
        s_code_count += 1
        total_bits += (bit_len + 1)
    return  signs, m_codes, bit_string, total_bits


def get_s_codes_qe(
    bit_string: str,
    brc: int,
) -> ([int], [int], [int], str):
    signs = []
    m_codes = []       
    s_code_count = 0
    total_bits = 0
    while s_code_count < 128:
        sign, bit_string = read_and_pop(bit_string, 1)
        m_code, bit_string, bit_len = huffman(bit_string, brc)
        signs.append(sign)
        m_codes.append(m_code)
        s_code_count += 1
        total_bits += (bit_len + 1)
    return signs, m_codes, bit_string, total_bits

In [415]:
s = '111111110010100111'
m, bs, bl = huffman(s, 2)
print(f"M_CODE: {m}")
print(f"BIT_STRING: {bs}")
print(f"BIT_LENGTH: {bl}")

M_CODE: 6
BIT_STRING: 110010100111
BIT_LENGTH: 6


In [416]:
class Packet:
    def __init__(
        self,
        primary_header,
        secondary_header,
        user_data_field
    ):
        self.primary_header   = primary_header
        self.secondary_header = secondary_header
        self.raw_user_data    = user_data_field

        self.version_number        = int(primary_header['packet_version_number'], 2)
        self.type                  = int(primary_header['packet_type'], 2)
        self.secondary_header_flag = int(primary_header['secondary_header_flag'], 2)
        self.process_id            = int(primary_header['process_id'], 2)
        self.process_category      = int(primary_header['process_category'], 2)
        self.sequence_flags        = int(primary_header['sequence_flags'])
        self.sequence_count        = int(primary_header['packet_sequence_count'], 2)
        self.packet_data_length    = int(primary_header['packet_data_length'], 2)
        self.user_data_length      = (self.packet_data_length + 1) - 62

        self.num_quads   = int(self.secondary_header['num_quadratures'], 2)
        self.num_samples = 2 * self.num_quads

        self.test_mode      = int(self.secondary_header['test_mode'], 2)
        self.baq_mode       = int(self.secondary_header['baq_mode'], 2)
        self.block_length   = int(self.secondary_header['baq_block_length'], 2)
        self.num_baq_blocks = int(np.ceil(2 * self.num_quads / 256))

        self._set_data_format()
        self._decode_user_data_field()


    def _set_num_words_type_c(self):
        num_words_ie = int(np.ceil(self.block_length * self.num_quads / 16))
        num_words_qe = int(np.ceil((self.block_length * self.num_quads + 8 * self.num_baq_blocks) / 16))
        self.num_words = (num_words_ie, num_words_qe)


    def _decode_ie(self, bit_string):
        ie_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc, bit_string = read_and_pop(bit_string, 3)
            if brc != '':
                brc = int(brc, 2)
                if brc <= 4:
                    signs, m_codes, bit_string, total_bits = get_s_codes(bit_string, brc)
                    self.ie_signs.append(signs)
                    self.ie_m_codes.append(m_codes)
                    self.brc.append(brc)
                else:
                    raise ValueError(f'Invalid BRC: {brc} at {i}')
            else:
                raise ValueError(f'No BRC!')
            ie_bits += (total_bits + 3)
        return bit_string, ie_bits

    def _decode_io(self, bit_string):
        io_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            signs, m_codes, bit_string, total_bits = get_s_codes(bit_string, brc)
            self.io_signs.append(signs)
            self.io_m_codes.append(m_codes)
            io_bits += total_bits
        return bit_string, io_bits


    def _decode_qe(self, bit_string):
        qe_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            threshold, bit_string = read_and_pop(bit_string, 8)
            signs, m_codes, bit_string, total_bits = get_s_codes_qe(bit_string, brc)
            self.qe_signs.append(signs)
            self.qe_m_codes.append(m_codes)
            self.thresholds.append([threshold])
            qe_bits += (total_bits + 8)
        return bit_string, qe_bits


    def _decode_qo(self, bit_string):
        qo_bits = 0
        total_bits = 0
        for i in range(self.num_baq_blocks):
            brc = self.brc[i]
            signs, m_codes, bit_string, total_bits = get_s_codes(bit_string, brc)
            self.qo_signs.append(signs)
            self.qo_m_codes.append(m_codes)
            qo_bits += total_bits
        return bit_string, qo_bits


    def _jump_to_next_word_boundary(self, bit_string, num_bits):
        offset = num_bits % 16
        if offset == 0:
            return bit_string
        next_word_boundary = 16 - offset
        filler, bit_string = read_and_pop(bit_string, next_word_boundary)
        return bit_string
    

    def _decode_type_c_data(self):    
        self._set_num_words_type_c()

        s_code_width = 0
        bit_string   = create_bit_string(self.raw_user_data)
        block_width  = (128 * self.block_length - 1) * self.num_baq_blocks

        self.brc        = []
        self.ie_signs   = []
        self.ie_m_codes = []
        self.io_signs   = []
        self.io_m_codes = []
        self.qe_signs   = []
        self.qe_m_codes = []
        self.qo_signs   = []
        self.qo_m_codes = []
        self.thresholds = []

        bit_string, ie_bits = self._decode_ie(bit_string)
        self._jump_to_next_word_boundary(bit_string, ie_bits)
        bit_string, io_bits = self._decode_io(bit_string)
        self._jump_to_next_word_boundary(bit_string, io_bits)
        bit_string, qe_bits = self._decode_qe(bit_string)
        self._jump_to_next_word_boundary(bit_string, qe_bits)
        bit_string, qo_bits = self._decode_qo(bit_string)
        self._jump_to_next_word_boundary(bit_string, qo_bits)

    def _decode_user_data_field(self):
        if self.data_format == 'A':
            pass
        elif self.data_format == 'B':
            pass
        elif self.data_format == 'C':
            pass
        elif self.data_format == 'D':
            self._decode_type_c_data()


    def _set_data_format(self):
        self.test_mode = int(self.secondary_header['test_mode'], 2)
        self.baq_mode  = int(self.secondary_header['baq_mode'], 2)
        if self.baq_mode == 0:
            if self.test_mode % 3 == 0:
                self.data_format = 'A'
            else:
                self.data_format = 'B'
        elif self.baq_mode in [3, 4, 5]:
            self.data_format = 'C'
        elif self.baq_mode in [12, 13, 14]:
            self.data_format = 'D'
        else:
            raise ValueError(f'No Data Format with BAQ Mode: {self.baq_mode} and Test Mode: {self.test_mode}')

In [464]:
def get_header_dict(header_bytes, header_bit_lengths, header_field_names):
    read_and_pop = lambda bit_string, bit_length: (bit_string[0:bit_length], bit_string[bit_length:])
    bit_string = ''
    for byte in header_bytes:
        bit_string += '{:08b}'.format(byte)

    header_value_array = []
    for bit_len in header_bit_lengths:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    index = 0
    header_dict = {}
    for field in header_field_names:
        if index >= len(header_field_names) or index >= len(header_value_array):
            raise ValueError('The number of field names and header values does not match.')
        header_dict[field] = header_value_array[index]
        index += 1

    return header_dict


def packet_generator(raw_data):
    secondary_header_length = 62
    while raw_data:
        primary_header_bytes = raw_data.read(6)
        primary_header = get_header_dict(primary_header_bytes, PRIMARY_HEADER, PRIMARY_HEADER_FIELDS)
        secondary_header_bytes = raw_data.read(62)
        secondary_header = get_header_dict(secondary_header_bytes, SECONDARY_HEADER, SECONDARY_HEADER_FIELDS)
        packet_data_length = primary_header['packet_data_length']
        user_data = None
        if packet_data_length:
            packet_data_length = int(packet_data_length, 2)
            user_data_length = (packet_data_length + 1) - secondary_header_length
            if user_data_length > 0:
                user_data = raw_data.read(user_data_length)
        yield Packet(
            primary_header = primary_header,
            secondary_header = secondary_header,
            user_data_field = user_data
        )

In [418]:
l0_data_file = 'sar_data/S1A_IW_RAW__0SDV_20240806T135224_20240806T135256_055093_06B68A_AE41.SAFE/s1a-iw-raw-s-vv-20240806t135224-20240806t135256-055093-06b68a.dat'

In [419]:
raw_data_file = open(l0_data_file, 'rb')

In [420]:
PacketGenerator = packet_generator(raw_data_file)

In [421]:
packet = next(PacketGenerator)

In [422]:
packet.primary_header

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010101',
 'packet_data_length': '0100100010111101'}

In [423]:
packet.secondary_header

{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001001101',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011011',
 'sc_data_word': '1011111011000011',
 'counter_service': '00000000000000111010101110010101',
 'pri_count': '00000000000000111011011010111000',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'sas_ssb_message': '011111000110000

In [424]:
next_packet = next(PacketGenerator)

In [425]:
next_packet.primary_header

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010110',
 'packet_data_length': '0100100010011001'}

In [426]:
next_packet.secondary_header

{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001110011',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011100',
 'sc_data_word': '1111000100001001',
 'counter_service': '00000000000000111010101110010110',
 'pri_count': '00000000000000111011011010111001',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'sas_ssb_message': '011111000110000

In [427]:
packet.secondary_header == next_packet.secondary_header

False

In [428]:
packet.num_samples == next_packet.num_samples

True

In [429]:
packet.data_format == next_packet.data_format == 'D'

True

In [430]:
packet.user_data_length

18560

In [431]:
next_packet.sequence_count

11158

In [432]:
next_packet.num_baq_blocks

94

In [433]:
len(packet.ie_m_codes)

94

In [434]:
ie_len = 0
io_len = 0
qe_len = 0
qo_len = 0
for arr in packet.ie_m_codes:
    ie_len += len(arr)
for arr in packet.io_m_codes:
    io_len += len(arr)
for arr in packet.qe_m_codes:
    qe_len += len(arr)
for arr in packet.qo_m_codes:
    qo_len += len(arr)
print(f"IE M_CODES: {ie_len}")
print(f"IO M_CODES: {io_len}")
print(f"QE M_CODES: {qe_len}")
print(f"QO M_CODES: {qo_len}")

IE M_CODES: 12032
IO M_CODES: 12032
QE M_CODES: 12032
QO M_CODES: 12032


In [462]:
np.asarray(packet.brc)

array([0, 1, 1, 1, 1, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3])

In [463]:
np.asarray(next_packet.brc)

array([0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3])

In [447]:
np.asarray(packet.raw_user_data)

array(b'\x11\x11\xa9\x01\x04\xdb\x86\xd0\x19.2\xd6\x1a\xb1\x8aQ\xa9\x1e\x8b\x18R\xd0\xac\x0f\xc1\xa1U\x92\xd8\x83\x94\xc6Il\xd0eD|\xb1\x81\x16$\x17\x12-\x90\x83*\x8d\x05\xc3\xb4\xbc\xa5s\x06\x972G6\x96!p\xe2\xd4\xeeh4\xfb\xa1\xcd\xe2&/\x19\x14l\x86o\xd2tJ\x80\x98\xc5\x12m1\x0b*E`S\x1a\xadH4h\x1d\x19\xb0\x07\n\xa7\x0bCCj\xbc\xd6\xb5;\x8d\xc229\x823?H8\x11\xcb\x1a\x18\x99\x140[e\xd2\x13\x1c\x10uJ\x99d,\xbcf\xd2J\x86\x16\xe8]2\x10\x88T\xb0\x1d\xdbl\x19\xc4\xc1\x84\xb8ZL\xdd\x02\x8d\xa7\x0cl\xdd\x9c\x12_\xef\x11$\xdc\xd0\x91CT\n\x89\xb91\xc53\xca2_\xd3\\\x06$\xb90\x9b\x9a#\x82\x8e`LGd\xb0\x9bup\x8eS\x1a9U\x90\xc1"l\t\tu\xea\x15\xcd0r\x1b\x82\xc0\xab\xa3\xb3\x15\x0e\xdb\x13|\xea<\xb8\xe6\xef.t\x97\xcc\x0c\xb1\x9c\xcf\xe5\xa4\'!;\x91#\xc0s;\\\xb9&\xf7\xc3b\xde\xc3\x07!\xc7i\x9cn\x04\xbd\xe7\xc7\xcayc\xf3\xc9`\x87Zcso3\x18\xe6\xc8\rxy#\xe24\x08\xdc\x08\x16e\xc1\xbc+\x1b\x9e/\x81]\x0b!X\x7f\xdc\xd0\t\x90\x8c\xf7d\xdc0,\xe5\xbfY"\xeb\t(\xc3o6\xf28\xac\x1aY.j\xcc\xe2\x1d\xc3~g\xac\xdc<H0k\xc0@\x

In [446]:
np.asarray(next_packet.raw_user_data)

array(b'\n\x0c\x962\xc2ivM\xf4\xb2b\xc3\xc6K\x1aM\xd4D\x94\x9bd%\xc1%jG7\x81#\xa5rjQk,\xb0f\xe3)\xa2j\xb2%`\xccr3-\x1al\xb8\xe7\xe6\xeac\x1df\x9b\x19Ml\x86\xbb\x91\x08\x1b\xac\xc0p(\xe6\xec\xc5@\xfe\xe12\xdc\xcc\x88\xcc\xa0\xday\xb2\xedu.\xe6\x08A\x9aU\x99f\xae\xb0\x01\x82\xe4\xa51\x07f\xa8n\x92\x14\xc1\xc0\xf0\xed\x9b\x8bI\x8d\x91%\x82\x00\xc4\xe3\xbb\x9e7}\\\xc0l1%em\x99\x16\x8bF,;\x80\xac8\xec\xd80C\xe2\x96\xba\xe7")g\xee\xabj\xf0\x08\xd80(\x00\xf5\x7f\x85\xc9N\x8d\xaaf*\x0b\xb3\x0c\x0b7\\!\xbbGW\x97\x12\x93\x17\x08\x00\xb1\x1cN\xecW\xc5\x143sz$7t\xc0f"\x81\x00\xc2\xdb\x86\x00\x0b\x19\tr\x0b\xdc\r\x11\x97a\xae\x9b\xb7\xa9\x85\x1e{\x0eK\x8d7\x840\xb8\xe9\xebo\x9c\xcct\x07\t,#2\x1d,\xff\xbb8f\xcfr7:\xe2Wv\xf0z\xcboM\x11\x91\xe6\x0c!\x1b\xb3\xf8\xf9\xd8f\xd6M#\xe5\x9a\xbe[\xc6b4\xb6\xb9\xef\r\xf0\xa3\x10\x1c\xe47K"\xd0\xcb$\xd6\x122\x1fn< \xe5F`N\xfb\xe7\xebv\xf3\x9f\xfayd:\xe5\xef\xa1\xa1rR\xf7\x91\xcb\xe6[\x93\x8d\xc2\xa4\xd8\x10/\x80X\xce\xe16F\xb76w\xc9\x16*\xd87!8X\x12\xcc";; \xed

In [448]:
create_bit_string(packet.raw_user_data)

'...'

In [449]:
create_bit_string(next_packet.raw_user_data)

'...'

In [450]:
next_packet.num_baq_blocks

94

In [459]:
np.asarray(packet.brc).shape

(94,)